<a href="https://colab.research.google.com/github/TV153/8th_sem_project/blob/main/FEASABILITY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyHSICLasso

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn import svm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import scipy.io
import pandas as pd

mat = scipy.io.loadmat('/content/drive/MyDrive/Project_8thSem/DATA/ALL-AML/data_set_ALL_AML_independent.csv/data_set_ALL_AML_independent.csv')
mat = {k:v for k, v in mat.items() if k[0] != '_'}
data = pd.DataFrame({k: pd.Series(v[0]) for k, v in mat.items()})

data.to_csv("example.csv")

ValueError: Unknown mat file type, version 108, 108

In [ ]:
from google.colab import files
files.download("example.csv")

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Project_8thSem/DATA/ALL-AML/data_set_ALL_AML_independent.csv/data_set_ALL_AML_independent.csv')


In [6]:
df


,Gene Description,Gene Accession Number,39,call,40,call.1,42,call.2,47,call.3,...,65,call.29,66,call.30,63,call.31,64,call.32,62,call.33
0,AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-342,A,-87,A,22,A,-243,A,...,-62,A,-58,A,-161,A,-48,A,-176,A
1,AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-200,A,-248,A,-153,A,-218,A,...,-198,A,-217,A,-215,A,-531,A,-284,A
2,AFFX-BioB-3_at (endogenous control),AFFX-BioB-3_at,41,A,262,A,17,A,-163,A,...,-5,A,63,A,-46,A,-124,A,-81,A
3,AFFX-BioC-5_at (endogenous control),AFFX-BioC-5_at,328,A,295,A,276,A,182,A,...,141,A,95,A,146,A,431,A,9,A
4,AFFX-BioC-3_at (endogenous control),AFFX-BioC-3_at,-224,A,-226,A,-211,A,-289,A,...,-256,A,-191,A,-172,A,-496,A,-294,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7124,PTGER3 Prostaglandin E receptor 3 (subtype EP3...,X83863_at,1074,A,67,A,893,P,722,A,...,707,A,423,A,809,A,466,A,551,A
7125,HMG2 High-mobility group (nonhistone chromosom...,Z17240_at,475,A,263,A,297,A,170,A,...,354,A,41,A,445,A,349,A,194,A
7126,RB1 Retinoblastoma 1 (including osteosarcoma),L49218_f_at,48,A,-33,A,6,A,0,A,...,-22,A,0,A,-2,A,0,A,20,A
7127,GB DEF = Glycophorin Sta (type A) exons 3 and ...,M71243_f_at,168,A,-33,A,1971,P,510,P,...,260,A,1777,P,210,A,284,A,379,A


In [7]:
labels = "/content/drive/MyDrive/Project_8thSem/DATA/ALL-AML/archive (5)/actual.csv"
test_path = "/content/drive/MyDrive/Project_8thSem/DATA/ALL-AML/archive (5)/data_set_ALL_AML_independent.csv"
train_path = "/content/drive/MyDrive/Project_8thSem/DATA/ALL-AML/archive (5)/data_set_ALL_AML_train.csv"

In [8]:
labels_df = pd.read_csv(labels, index_col = 'patient')

X_test = pd.read_csv(test_path)
X_train = pd.read_csv(train_path)

In [9]:
# extract only call columns, in order to process them and potentially use them as features
ds_test = [col for col in X_test.columns if 'call' in col]
X_test_call = X_test[ds_test]

# drop call columns from X_test
X_test.drop(ds_test, axis = 1, inplace = True)

# extract only call columns, in order to process them and potentially use them as features
ds_train = [col for col in X_train.columns if 'call' in col]
X_train_call = X_train[ds_train]

# drop call columns from X_test
X_train.drop(ds_train, axis = 1, inplace = True)


In [10]:
X_train_call = X_train_call.replace("A", 0)
X_train_call = X_train_call.replace("P", 1)
X_train_call = X_train_call.replace("M", 2)

X_test_call = X_test_call.replace("A", 0)
X_test_call = X_test_call.replace("P", 1)
X_test_call = X_test_call.replace("M", 2)

In [11]:
# add rownames which include the "Gene Accession number" + "Call"
rownames = "Call " + X_train.iloc[:,1]
X_train_call.index = rownames
X_test_call.index = rownames

# extract ids for patients
ids_train = list(X_train.columns[2:])
ids_test = list(X_test.columns[2:])

X_train_call.columns = ids_train
X_test_call.columns = ids_test

In [12]:
X_train_call = X_train_call.T
X_train_call.head()

X_test_call = X_test_call.T
X_test_call.head()

Gene Accession Number,Call AFFX-BioB-5_at,Call AFFX-BioB-M_at,Call AFFX-BioB-3_at,Call AFFX-BioC-5_at,Call AFFX-BioC-3_at,Call AFFX-BioDn-5_at,Call AFFX-BioDn-3_at,Call AFFX-CreX-5_at,Call AFFX-CreX-3_at,Call AFFX-BioB-5_st,...,Call U48730_at,Call U58516_at,Call U73738_at,Call X06956_at,Call X16699_at,Call X83863_at,Call Z17240_at,Call L49218_f_at,Call M71243_f_at,Call Z78285_f_at
39,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
42,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,1,0
47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
48,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


In [13]:
train_ids = X_train_call.index
train_ids = list(map(int, train_ids))

test_ids = X_test_call.index
test_ids = list(map(int, test_ids))

train_labels = []
test_labels = []

for i in train_ids:
  train_labels.append(labels_df.iloc[i-1,0])

for j in test_ids:
  test_labels.append(labels_df.iloc[j-1,0])


train_labels = pd.concat([pd.DataFrame(train_ids), pd.DataFrame(train_labels)], axis=1, ignore_index = False)
train_labels.columns = ["patient", "cancer"]

test_labels = pd.concat([pd.DataFrame(test_ids), pd.DataFrame(test_labels)], axis=1, ignore_index = False)
test_labels.columns = ["patient", "cancer"]

# extract only the labels in the y variables

y_train = train_labels.iloc[:,1]
y_test = test_labels.iloc[:,1]

# replace ALL labels with 0 and AML labels with 1

y_train = y_train.replace("ALL", 0)
y_train = y_train.replace("AML", 1)
y_train = list(y_train)


y_test = y_test.replace("ALL", 0)
y_test = y_test.replace("AML", 1)
y_test = list(y_test)

In [14]:
gene_names = X_train.iloc[:,1]
X_train = X_train.iloc[:,2:]
X_train = X_train.T
X_train.columns = gene_names
X_train.index = train_ids

X_test = X_test.iloc[:,2:]
X_test = X_test.T
X_test.columns = gene_names
X_test.index = test_ids
X_test.head()

Gene Accession Number,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,AFFX-BioDn-3_at,AFFX-CreX-5_at,AFFX-CreX-3_at,AFFX-BioB-5_st,...,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at
39,-342,-200,41,328,-224,-427,-656,-292,137,-144,...,277,1023,67,214,-135,1074,475,48,168,-70
40,-87,-248,262,295,-226,-493,367,-452,194,162,...,83,529,-295,352,-67,67,263,-33,-33,-21
42,22,-153,17,276,-211,-250,55,-141,0,500,...,413,399,16,558,24,893,297,6,1971,-42
47,-243,-218,-163,182,-289,-268,-285,-172,52,-134,...,174,277,6,81,2,722,170,0,510,-73
48,-130,-177,-28,266,-170,-326,-222,-93,10,159,...,233,643,51,450,-46,612,370,29,333,-19


In [15]:

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train= scaler.transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = gene_names
X_train.index = train_ids
X_train.head()

Gene Accession Number,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,AFFX-BioDn-3_at,AFFX-CreX-5_at,AFFX-CreX-3_at,AFFX-BioB-5_st,...,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at
1,0.531440,0.566775,0.435315,0.289720,0.502041,0.365354,0.755011,0.653759,0.915068,0.542526,...,0.475460,0.200840,0.133838,0.141710,0.457317,0.282075,0.269663,0.521212,0.060407,0.438462
2,0.683570,0.827362,0.534965,0.745327,0.565306,0.614173,0.165924,0.671982,0.501370,0.372423,...,0.426380,0.428971,0.358586,0.162987,0.579268,0.276887,0.237828,0.369697,0.024413,0.615385
3,0.811359,0.905537,0.000000,0.806075,0.336735,0.220472,0.570156,0.218679,0.789041,0.000000,...,0.874233,0.682295,0.532828,0.052991,1.000000,0.444811,0.689139,0.551515,0.071987,0.407692
4,0.691684,0.693811,1.000000,0.112150,0.248980,0.322835,0.709354,0.478360,0.358904,0.317010,...,0.644172,0.427572,1.000000,0.055399,0.012195,0.203774,0.120787,0.000000,0.040063,0.023077
5,0.750507,0.657980,0.403846,0.476636,0.634694,0.796850,0.537862,0.776765,0.416438,0.601804,...,0.386503,0.297411,0.593434,0.187876,0.524390,0.025943,0.255618,0.387879,0.018153,0.530769


In [16]:
X_train_2 = X_train.T

In [17]:
X_train_2.head(5)

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,28,29,30,31,32,33
Gene Accession Number,,,,,,,,,,,,,,,,,,,,,
AFFX-BioB-5_at,0.531440,0.683570,0.811359,0.691684,0.750507,0.685598,0.819473,0.127789,0.975659,0.787018,...,0.979716,0.533469,0.914807,0.819473,0.957404,0.995943,0.320487,0.900609,0.713996,0.691684
AFFX-BioB-M_at,0.566775,0.827362,0.905537,0.693811,0.657980,0.788274,0.596091,0.218241,0.651466,0.723127,...,0.739414,0.244300,1.000000,0.612378,0.687296,0.693811,0.439739,0.905537,0.807818,0.459283
AFFX-BioB-3_at,0.435315,0.534965,0.000000,1.000000,0.403846,0.912587,0.952797,0.548951,0.722028,0.610140,...,0.437063,0.774476,0.753497,0.534965,0.318182,0.540210,0.370629,0.622378,0.472028,0.414336
AFFX-BioC-5_at,0.289720,0.745327,0.806075,0.112150,0.476636,0.250000,0.212617,0.079439,0.710280,0.595794,...,0.392523,0.827103,0.843458,1.000000,0.647196,0.535047,0.813084,0.621495,0.855140,0.871495
AFFX-BioC-3_at,0.502041,0.565306,0.336735,0.248980,0.634694,0.548980,0.289796,0.000000,0.675510,0.740816,...,0.334694,0.677551,0.295918,0.442857,0.714286,1.000000,0.820408,0.355102,0.720408,0.273469


In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

from future import standard_library

from pyHSICLasso import HSICLasso

import scipy.io as sio

standard_library.install_aliases()


def main():

    hsic_lasso = HSICLasso()
    data = X_train
    X = data['X'].transpose()
    Y = data['Y'][0]
    featname = [ 'Feat%d' % x for x in range(1, X.shape[1] + 1) ]

    hsic_lasso.input(X,Y,featname=featname)
    hsic_lasso.regression(5,covars=X)
    hsic_lasso.dump()
    hsic_lasso.plot_path()
    hsic_lasso.save_param()

if __name__ == "__main__":
    main()

KeyError: 'X'